In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
import numpy as np 
import pandas as pd
from typing import Tuple, Optional, Union
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.linear_model import LinearRegression
from skforecast.utils import check_y
from skforecast.utils import transform_series
from skforecast.utils import preprocess_y
import time
import numba as nb
import warnings
import sklearn
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.exceptions import NotFittedError
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

# Create lags

In [4]:
def get_feature_importance(
    self
) -> pd.DataFrame:
    """      
    Return feature importance of the regressor stored in the
    forecaster. Only valid when regressor stores internally the feature
    importance in the attribute `feature_importances_` or `coef_`.

    Parameters
    ----------
    self

    Returns
    -------
    feature_importance : pandas DataFrame
        Feature importance associated with each predictor.

    """

    if self.fitted == False:
        raise sklearn.exceptions.NotFittedError(
            ("This forecaster is not fitted yet. Call `fit` with appropriate "
             "arguments before using `get_feature_importance()`.")
        )

    if isinstance(self.regressor, sklearn.pipeline.Pipeline):
        estimator = self.regressor[-1]
    else:
        estimator = self.regressor

    try:
        feature_importance = pd.DataFrame({
                                    'feature': self.X_train_col_names,
                                    'importance' : estimator.feature_importances_
                                })
    except:   
        try:
            feature_importance = pd.DataFrame({
                                        'feature': self.X_train_col_names,
                                        'importance' : estimator.coef_
                                    })
        except:
            warnings.warn(
                f"Impossible to access feature importance for regressor of type {type(estimator)}. "
                f"This method is only valid when the regressor stores internally "
                f"the feature importance in the attribute `feature_importances_` "
                f"or `coef_`."
            )

            feature_importance = None

    return feature_importance



def get_feature_importance_new(
    self
) -> pd.DataFrame:

    if self.fitted == False:
        raise sklearn.exceptions.NotFittedError(
            ("This forecaster is not fitted yet. Call `fit` with appropriate "
             "arguments before using `get_feature_importance()`.")
        )

    if isinstance(self.regressor, sklearn.pipeline.Pipeline):
        estimator = self.regressor[-1]
    else:
        estimator = self.regressor

    if hasattr(estimator, 'feature_importances_'):
        feature_importance = estimator.feature_importances_
    elif hasattr(estimator, 'coef_'):
        feature_importance = estimator.coef_
    else:
        warnings.warn(
            (f"Impossible to access feature importance for regressor of type {type(estimator)}. "
             f"This method is only valid when the regressor stores internally "
             f"the feature importance in the attribute `feature_importances_` "
             f"or `coef_`.")
        )

        feature_importance = None

    if feature_importance is not None:
        feature_importance = pd.DataFrame({
                                 'feature': self.X_train_col_names,
                                 'importance': feature_importance
                             })

    return feature_importance


In [12]:
model = RandomForestRegressor(n_estimators=1, max_depth=2, random_state=123)

forecaster = ForecasterAutoreg(
                regressor = make_pipeline(StandardScaler(), model),
                lags      = 3
                )
# forecaster = ForecasterAutoreg(
#                 regressor = model,
#                 lags      = 3
#                 )
forecaster.fit(y=pd.Series(np.arange(5)))

if isinstance(forecaster.regressor, sklearn.pipeline.Pipeline):
    estimator = forecaster.regressor[-1]
else:
    estimator = forecaster.regressor

if hasattr(estimator, 'coef_'):
    print(1)
elif hasattr(estimator, 'feature_importances_'):
    print(2)
else:
    print(0)

2


In [13]:
self = ForecasterAutoreg(
                    regressor = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=1, max_depth=2, random_state=123)),
                    lags      = 3
                    )
self = ForecasterAutoreg(regressor=LinearRegression(), lags=3)
y = pd.Series(np.random.normal(size=100), name='y')
self.fit(y=y)

start = time.time()
X = get_feature_importance(self)
print(f"Time taken get_feature_importance: {time.time() - start}")
print(X)



start = time.time()
X = get_feature_importance_new(self)
print(f"Time taken get_feature_importance_new: {time.time() - start}")
print(X)

Time taken get_feature_importance: 0.0
  feature  importance
0   lag_1   -0.190839
1   lag_2   -0.137537
2   lag_3   -0.034713
Time taken get_feature_importance_new: 0.0
  feature  importance
0   lag_1   -0.190839
1   lag_2   -0.137537
2   lag_3   -0.034713


In [10]:
%timeit -r 4 -n 10000 get_feature_importance(self)

74.2 µs ± 1.58 µs per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [11]:
%timeit -r 4 -n 10000 get_feature_importance_new(self)

71.9 µs ± 1.46 µs per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [7]:
X

,feature,importance
0,lag_1,0.064216
1,lag_2,0.146984
2,lag_3,-0.036794


In [8]:
# Profiling `_create_lags` for different length of series and number of lags
# ======================================================================================
series_length = np.linspace(1000, 1000000, num=5, dtype=int)
n_lags = [5, 10, 50, 100, 200]
results = {}

for lags in n_lags:
    execution_time = []
    self = ForecasterAutoreg(regressor=LinearRegression(), lags=lags)

    for n in series_length:
        y = np.random.normal(size=n)
        tic = time.perf_counter()
        _ = _create_lags(self, y=y)
        toc = time.perf_counter()
        execution_time.append(toc-tic)

    results[lags] = execution_time

results = pd.DataFrame(
              data =  results,
              index = series_length
          )

results

,5,10,50,100,200
1000,0.000242,0.001026,0.002356,0.008464,0.015897
250750,0.003495,0.009611,0.193564,0.230684,0.271816
500500,0.009737,0.031638,0.393094,0.566737,1.101087
750250,0.014285,0.052606,0.469482,0.834637,1.709996
1000000,0.023364,0.070215,0.580613,1.130334,2.026127


In [7]:
# Profiling `_create_lags_new` for different length of series and number of lags
# ======================================================================================
series_length = np.linspace(1000, 1000000, num=5, dtype=int)
n_lags = [5, 10, 50, 100, 200]
results = {}

for lags in n_lags:
    execution_time = []
    self = ForecasterAutoreg(regressor=LinearRegression(), lags=lags)

    for n in series_length:
        y = np.random.normal(size=n)
        tic = time.perf_counter()
        _ = _create_lags_new(self, y=y)
        toc = time.perf_counter()
        execution_time.append(toc-tic)

    results[lags] = execution_time

results = pd.DataFrame(
              data =  results,
              index = series_length
          )

results

,5,10,50,100,200
1000,0.000053,0.000187,0.000189,0.000180,0.000190
250750,0.000043,0.000054,0.000025,0.000028,0.000020
500500,0.000178,0.000095,0.000105,0.000073,0.000061
750250,0.000150,0.000146,0.000135,0.000137,0.000106
1000000,0.000190,0.000205,0.000179,0.000176,0.000158


In [7]:
y = pd.Series(np.arange(10), name='y', dtype=float)
exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})

forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=5, max_depth=2, random_state=123),
                    lags      = 3
                )
forecaster.fit(y=y, exog=exog)

results = forecaster.get_feature_importance()
results

,feature,importance
0,lag_1,0.206222
1,lag_2,0.200000
2,lag_3,0.207366
3,exog_1,0.213725
4,exog_2,0.172686


In [12]:
forecaster.get_feature_importance()

,feature,importance
0,lag_1,0.206222
1,lag_2,0.200000
2,lag_3,0.207366
3,exog_1,0.213725
4,exog_2,0.172686
